In [81]:
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import time
from sklearn.model_selection import KFold
import warnings
from sklearn.model_selection import cross_val_score

In [2]:
start_time = time.time()
df = pd.read_csv('C:\\users\\user\\cc_sample.txt', sep=';',index_col=False, decimal=',') 
print(-start_time + time.time()," sec")

2.7400450706481934  sec


In [32]:
#Get Dataframe as input, return DataFrame with filled nulls
def fill_null(x,s):
    for i in x.columns:
        x[i] = x[i].fillna(s)
    return x

#Check type of variable and return numeric or cat
def check_type(x):
    from pandas.api.types import is_string_dtype
    from pandas.api.types import is_numeric_dtype   
    #Удаляем пустые значения
    x = x[x.notnull()]
    #Если число различных значений меньше 4, то тип-категориальный
    if x.nunique()<=4: return 'cat'
    elif is_numeric_dtype(x): return 'numeric'
    else: return 'cat'
    
#Input: DataFrame
#Output: DataFrame with one-hot variables
def cat_to_one_hot(x):
    for col in x.columns:
        if check_type(x[col])=='cat':
            tmp = pd.get_dummies(x[col],prefix=x[col].name,drop_first=True)
            for i in tmp.columns:
                x[i] = tmp[i]
            del x[col]
    return x

def gini(model,x,y):
    gini =  2*roc_auc_score(y,model.predict_proba(x)[:,1])-1
   # print('GINI = ',2*roc_auc_score(y,model.predict_proba(x)[:,1])-1)   
    return gini
    

In [4]:
df = fill_null(df,-1)
df = cat_to_one_hot(df)

In [139]:
warnings.simplefilter('ignore', DeprecationWarning)
start_time = time.time()
x_sample=df.copy()
x_sample = x_sample.drop(['CONTRACT_SRC_CODE','SCORE_FINAL','BAD_12_FLAG90_1'], axis=1)
y = df["BAD_12_FLAG90_1"][df['BAD_12_FLAG90_1'].notnull()] 

xgb_model = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                               gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=4,
                               min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
                               objective='binary:logistic', reg_alpha=30, reg_lambda=1,
                               scale_pos_weight=1, seed=42, silent=True, subsample=1,tree_method='gpu_hist')
kfold = KFold(n_splits=3, shuffle=True)
clf = GridSearchCV(xgb_model,{'learning_rate': [0.1]}, 
                        verbose=2, 
                        scoring='roc_auc',
                        cv=kfold,
                        n_jobs=-1)
clf.fit(x_sample,y)
print(-start_time + time.time()," sec")


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   25.3s finished


40.08197855949402  sec


In [141]:
warnings.simplefilter('ignore', FutureWarning)
res = pd.DataFrame(clf.cv_results_)
res["dev_gini"] = 2*res["mean_train_score"]-1
res["val_gini"] = 2*res["mean_test_score"]-1
res[["params","dev_gini","val_gini"]]


,params,dev_gini,val_gini
0,{'learning_rate': 0.1},0.702631,0.656443
